In [1]:

import deap as dp
from Tools import * 
import random


In [2]:
Detailed_gas = ct.Solution("detailed.yaml")
Reduced_gas = ct.Solution("reduced.yaml")
pressure = np.linspace(1,1,1).tolist()
temperature = np.linspace(1500,2000,2).tolist()
phi = np.linspace(0.7,1.5,2).tolist()
mixture = [0.85]
Targets = ["H2", "NH3", "O2", "OH","NO","NH","NH2","NNH"]
Non_Target = ['N2', 'H', 'O', 'H2O', 'HO2', 'N', 'N2H3', 'N2H2', 'HNO', 'NO2', 'N2O']

param = ["shift","interpol","scaler"]

fuel1 = "NH3"
fuel2 ="H2"
oxidizer = 'O2:0.21, N2:0.79' 
tmax = 0.2
dt= 1e-6

/tmp/ipykernel_1014878/517391569.py:1: UserWarning: NasaPoly2::validate: 
For species OHV, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  53.62056162666667
	Value computed using high-temperature polynomial: 53.5841554314

  Detailed_gas = ct.Solution("detailed.yaml")


In [3]:
# Get info from det and red 
idx_target_det = spcs_name_idx(Detailed_gas,Targets)
idx_target_red = spcs_name_idx(Reduced_gas,Targets)

idx_non_target_det = spcs_name_idx(Detailed_gas,Non_Target)
idx_non_target_red = spcs_name_idx(Reduced_gas,Non_Target)


# Calcul Data Origin from Detailed 
case = generate_test_cases_bifuel(temperature,pressure,phi,mixture)
print(case)

[(101325.0, 1500.0, 0.7, 0.85), (101325.0, 1500.0, 1.5, 0.85), (101325.0, 2000.0, 0.7, 0.85), (101325.0, 2000.0, 1.5, 0.85)]


In [4]:
Time_det , Temp_det, Y_Target_det,Y_Non_Target_det = Sim0D_launch(Detailed_gas,fuel1,fuel2,oxidizer,case,idx_target_det,idx_non_target_det,dt,tmax)

N_Time_det , N_Temp_det, N_Y_Target_det,N_Y_Non_Target_det, N_Scaler_Target_det,N_Scaler_Non_Target_det,N_AI_delay_det= Change_detailed(Time_det,Temp_det,Y_Target_det,Y_Non_Target_det,case,Targets,Non_Target,param)


In [5]:
# No transform 
Time_red , Temp_red, Y_Target_red,Y_Non_Target_red = Sim0D_launch(Reduced_gas,fuel1,fuel2,oxidizer,case,idx_target_red,idx_non_target_red,dt,tmax) 

#transform (Shift, Interp, Scal)
N_Time_red , N_Temp_red, N_Y_Target_red,N_Y_Non_Target_red,N_AI_delay_red =Change_Reduced(Time_red,Temp_red,Y_Target_red,Y_Non_Target_red,case,Targets,Non_Target,N_Time_det,N_Scaler_Target_det,N_Scaler_Non_Target_det,param)


OptiSMOKE++


In [6]:
def fit_OptimSmoke(Y_exp, Y_sim,case):
    fit =[]
    fac = []
    for c in range(len(case)) :
        fit.append(np.sum(np.sum((np.array(Y_exp[c]) -np.array(Y_sim[c]))**2,axis = 1)))
        fac.append(np.shape(Y_exp[c])[1])
    return np.sum(np.array(fit)/np.array(fac))


In [7]:
err = fit_OptimSmoke(N_Y_Target_det,N_Y_Target_red,case)
print(err)

11.681394856386255


PYOPTMECH

In [48]:
def fit_pyoptmec_test(Y_t_red,Y_t_det,time_det,Y_nt_red,Y_nt_det,temp_red,temp_det,ai_det,ai_red,case) : 
    F1_m =[]
    F2_m =[]
    F3_m =[] 
    F4_m = []
    for c in range(len(case)) : 
        # Equation F1_m
        F1_m.append( np.sum([
            (np.trapezoid(np.abs(np.array(Y_t_red[c]) - np.array(Y_t_det[c])), np.array(time_det[c])) / np.trapezoid(np.abs(np.array(Y_t_det[c])), np.array(time_det[c])))**2
        ]))

        # Equation F2_m
        F2_m.append( np.sum([
            ((np.max(Y_nt_red[c],axis=1)[j] - np.max(Y_nt_det[c],axis=1)[j]) / np.max(Y_nt_det[c],axis=1)[j])**2 for j in range(len(np.max(Y_nt_det[c],axis=1)))
        ]))

        # Equation F3_m
        F3_m .append( (np.trapezoid(np.abs(np.array(temp_red[c]) - np.array(temp_det[c])), np.array(temp_det[c])) / np.trapezoid(np.abs(np.array(temp_det[c])), np.array(temp_det[c])))**2)

        # Equation F4_m
        F4_m.append(((ai_red[c] - ai_det[c]) / ai_det[c])**2)
        
    weight = [1,1,1,1]
    print(F1_m)
    print(F2_m)
    print(F3_m)
    print(F4_m)
    _err = (
                weight[0] * F1_m
                + weight[1] * F2_m
                + weight[2] * F3_m
                + weight[3] * F4_m
            )

    err = np.linalg.norm(_err)
    return err

In [49]:
err = fit_pyoptmec_test(N_Y_Target_red,N_Y_Target_det,N_Time_det,N_Y_Non_Target_red,N_Y_Non_Target_det,N_Temp_red,N_Temp_det,N_AI_delay_det,N_AI_delay_red,case) 
print(err)


[np.float64(0.04956139296523768)]
[np.float64(1.5723722643258684)]
[np.float64(0.0004083997861453475)]
[0.003002439482079185]
1.5731560795824242
